In [3]:
import numpy as np
import re
import fasttext
import time
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import pandas as pd
import subprocess
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
f_news = []

with open('datasets/news_fasttext_train.txt') as f:
    for line in f:
        f_news.append(f.readline())

print(f_news[-2].split('\t')[0].replace('\xa0', '').replace('  ', '').split('\u3000 \u3000'))

['数码 通及 合 和 实业 回购 股份 ', ' 新浪 财经 讯 12 月 01 日 消息 ， 数码 通 ( 00315 - HK ) 申报 回购 33 万股 ， 每股 作价 5.12 元至 5.2 元 ， 涉资 170.41 万元 。 ', ' 合 和 实业 ( 00054 - HK ) 申报 回购 1 , 500 股 ， 每股 作价 24.3 元 ， 涉 3.65 万元 。已有 _ COUNT _ 条 评论 我要 评论']


In [7]:
# 讲corpus按句号分行
with open('datasets/news_fasttext_train.txt') as origin:
    with open('datasets/test.txt', 'w') as f:
    #with open('datasets/news_raw_nolabel.txt', 'w') as f:
        for line in origin:
            temp = line.split('\t')[0].replace('\xa0', '').replace('  ', '').split('\u3000 \u3000')
            for i in range(len(temp)):
                temp[i] = temp[i].replace('\u3000', '')
                f.write("{}\n".format(temp[i]))

In [9]:
# 讲非中文字符去除
def filter(s):
    """
    Output only Chinese characters
    """
    filtrate = re.compile(u'[^\u4E00-\u9FA5]')
    res = filtrate.sub(r' ', s)
    return res

with open('datasets/test.txt') as origin:
    with open('datasets/news_pure_nolabel.txt', 'w') as f:
        for line in origin:
            temp = filter(line).replace('  ', '')
            f.write("{}\n".format(temp))

In [10]:
tokens = 0
i = 0
with open('datasets/news_pure_nolabel.txt', 'r') as f:
    for line in f:
        tokens += len(line.split())

print("Total number of tokens : " + str(tokens))

Total number of tokens : 47702885


In [2]:
# 添加<S>和</S>两个字符

with open('pre_trained_vectors/raw_300/ft_pure_skipgram_300.vec', 'r') as f:
    with open('pre_trained_vectors/raw_300/ft_5_vocab.txt', 'w+') as file:
        file.write('<S>\n</S>\n')
        _ = f.readline()
        for line in f:
            temp = line.split()
            file.write('{}\n'.format(temp[0]))            

In [3]:
# 添加<S>、</S>、<UNK>三个字符
with open('pre_trained_vectors/raw_300/glove_pure_5_vocab_original.txt', 'r') as f:
    with open('pre_trained_vectors/raw_300/glove_5_vocab.txt', 'w+') as file:
        file.write('<S>\n</S>\n<UNK>\n')
        for line in f:
            temp = line.split()
            file.write('{}\n'.format(temp[0]))  

Transfer the txt vectors to hdf5

In [4]:
import h5py
import numpy as np

n_token = 0
with open('/home/chenxin/WordEmbedding/pre_trained_vectors/raw_300/ft_5_vocab.txt', 'r') as f:
    for line in f:
        n_token += 1
        
embeddings = np.zeros((n_token, 300))
i = 2
with open('/home/chenxin/WordEmbedding/pre_trained_vectors/raw_300/ft_pure_skipgram_300.vec', 'r') as f:
    _ = f.readline()
    for line in f:
        temp = line.split()
        vec = np.array(temp[1:])
        embeddings[i] = vec
        i += 1


In [5]:
embeddings.shape

(182713, 300)

In [6]:
with h5py.File('vocab_embedding.hdf5', 'w') as fout:
    ds = fout.create_dataset(
        'embedding', embeddings.shape, dtype='float32', data=embeddings
    )